<center>
<h1><b>Lab 2</b></h1>
<h1>PHYS 580 - Computational Physics</h1>
<h2>Professor Molnar</h2>
</br>
<h3><b>Ethan Knox</b></h3>
<h4>https://www.github.com/ethank5149</h4>
<h4>ethank5149@gmail.com</h4>
</br>
</br>
<h3><b>September 11, 2020</b></h3>
</center>

### Imports

In [1]:
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from functools import partial

### Euler Step

In [31]:
def euler_step(f, y, t, dt):
    assert(type(f)==callable)
    print(f'euler_step: {type(f)}, {type(y)}, {type(t)}, {type(dt)}')
    return y + f(t, y) * dt

### Parameters

In [32]:
g = 9.81
v0 = 700
dt = 0.01
dtheta = 0.01
b2_m = 4.0e-5
y0scale = 1.0e4
alpha = 6.5e-3
gamma = 1.4
T_grd = 293

theta0 = 5
theta1 = 85
ntheta = round((theta1 - theta0) / dtheta)
termination_condition = lambda _t, _y : _t > 0. and _y[1] <= 0.
step = euler_step

### DSolve Function

In [42]:
def dsolve(fun, t, y0):
    assert(type(fun)==callable)
    t = np.asarray(t)  # Ensure t is a Numpy array
    y = np.zeros((np.size(t), np.size(y0)))  # Create our output data container
    y[0] = y0  # Set initial condition

    terminated_at = -1  # Index of the terminated point

    for i in range(np.size(t)-1):
        y[i+1] = step(fun, y[i], t[i], t[i+1] - t[i])  # Step forward

        if termination_condition(t[i], y[i]):  # Check termination condition
            terminated_at = i  # Set termination point
            break
    
    return t[:terminated_at], y[:terminated_at, :]

### Center Difference Function

In [43]:
def center_difference(f, x_i):
    return (f(x_i + 0.5 * dtheta) - f(x_i - 0.5 * dtheta)) / dtheta

### Bisection Algorithm

In [54]:
def bisection(f,a,b,N):
    if f(a)*f(b) >= 0:
        return None
    a_n = a
    b_n = b
    for n in range(1,N+1):
        m_n = (a_n + b_n)/2
        f_m_n = f(m_n)
        if f(a_n)*f_m_n < 0:
            a_n = a_n
            b_n = m_n
        elif f(b_n)*f_m_n < 0:
            a_n = m_n
            b_n = b_n
        elif f_m_n == 0:
            return m_n
        else:
            return None
    return (a_n + b_n)/2

## Drag Forces

In [55]:
def df_isothermal_drag(x,  y):
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v * np.exp(-y[1] / y0scale)
    return np.asarray([y[2], y[3], drag_factor * y[2], -g + drag_factor * y[3]])


def df_adiabatic_drag(x, y):
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v * (1 - alpha * y[1]/T_grd)**(1/ (gamma-1))
    return np.asarray([y[2], y[3], drag_factor * y[2], -g + drag_factor * y[3]])


def df_no_drag(x, y):
    return np.asarray([y[2], y[3], 0, -g])


def df_constant_drag(x, y):
    v = np.sqrt(y[2]**2 + y[3]**2)
    drag_factor = -b2_m * v
    return np.asarray([y[2], y[3], drag_factor * y[2], -g + drag_factor * y[3]])

## "Shooting" Function

In [56]:
def shoot(theta, forcing):
    assert(type(forcing)==callable)
    # Convert initial conditions into an initial velocity
    vx0 = v0 * np.cos(theta / 180. * np.pi)  
    vy0 = v0 * np.sin(theta / 180. * np.pi)
    y0 = np.asarray([0., 0., vx0, vy0])  # Set initial condition

    theoretical_max_range = v0**2 / g   # max range of shell, in vaccuum (for automatic horizontal plot range)
    theoretical_max_flight_time = theoretical_max_range / y0[2]  # flight time at maxr, in vacuum (for atomatic calculation end time)

    nsteps = round(theoretical_max_flight_time / dt)   # time steps
    t = np.linspace(0, theoretical_max_flight_time, int(nsteps))
    
    t, y_soln = dsolve(forcing, t, y0)
    y_soln = y_soln.T

    x, y, dx, dy = y_soln[0], y_soln[1], y_soln[2], y_soln[3]

    max_range = (y[-1] * x[-2] - y[-2] * x[-1] ) / (y[-1] - y[-2])
    x[-1] = max_range
    y[-1] = 0.
    
    max_height_index = np.argmax(y)
    max_height = y[max_height_index]
    x_at_max_height = x[max_height_index]

    return x, y, dx, dy, x_at_max_height, max_height, max_range

In [57]:
def max_range(theta, forcing):
    assert(type(forcing)==callable)
    soln = shoot(theta, forcing)
    return soln[6]

def d_max_range(theta, dtheta, forcing):
    assert(type(forcing)==callable)
    return center_difference(partial(max_range, forcing=forcing), theta)

## Sweep Optimum Angle

In [59]:
function_to_zero = partial(d_max_range, dtheta=dtheta, forcing=df_no_drag)
no_drag_optimum_angle = bisection(function_to_zero, 5, 85, 100)
print(no_drag_optimum_angle)

function_to_zero = partial(d_max_range, dtheta=dtheta, forcing=df_constant_drag)
constant_drag_optimum_angle = bisection(function_to_zero, 5, 85, 100)
print(constant_drag_optimum_angle)

function_to_zero = partial(d_max_range, dtheta=dtheta, forcing=df_adiabatic_drag)
adiabatic_drag_optimum_angle = bisection(function_to_zero, 5, 85, 100)
print(adiabatic_drag_optimum_angle)

function_to_zero = partial(d_max_range, dtheta=dtheta, forcing=df_isothermal_drag)
isothermal_drag_optimum_angle = bisection(function_to_zero, 5, 85, 100)
print(isothermal_drag_optimum_angle)

AssertionError: 

In [ ]:
no_drag = shoot(no_drag_optimum_angle, df_no_drag)
constant_drag = shoot(constant_drag_optimum_angle, df_constant_drag)
adiabatic_drag = shoot(adiabatic_drag_optimum_angle, df_adiabatic_drag)
isothermal_drag = shoot(isothermal_drag_optimum_angle, df_isothermal_drag)

## Plotting

### Sweeping Optimum Launch Angle

In [ ]:
fig = plt.figure(figsize=(16,9), constrained_layout=True)
plt.xlabel('Angle [degrees]')
plt.ylabel('Range [m]')
plt.suptitle("Ranges")
plt.plot(no_drag[0], no_drag[1], label = "No Drag")
plt.plot(constant_drag[0], constant_drag[1], label = "Constant Drag")
plt.plot(adiabatic_drag[0], adiabatic_drag[1], label = "Adiabatic Drag")
plt.plot(isothermal_drag[0], isothermal_drag[1], label = "Isothermal Drag")
plt.legend()
plt.grid()
plt.show()

In [ ]:
from scipy.integrate import solve_ivp
from scipy.misc import derivative
from scipy.optimize import root_scalar

In [ ]:
terminate = terminate_condition
terminate.terminal = True
force = df_adiabatic_drag

def get_range(theta, force, terminate):
    vx0 = v0 * np.cos(theta / 180. * np.pi)  
    vy0 = v0 * np.sin(theta / 180. * np.pi)
    y0 = np.asarray([0., 0., vx0, vy0])  # Set initial condition
    
    theoretical_max_range = v0**2 / g   # max range of shell, in vaccuum (for automatic horizontal plot range)
    theoretical_max_flight_time = theoretical_max_range / y0[2]  # flight time at maxr, in vacuum (for atomatic calculation end time)
    
    soln = solve_ivp(df_no_drag, (0, theoretical_max_flight_time), y0, events=terminate)
    return (soln.y[1,-1] * soln.y[0,-2] - soln.y[1,-2] * soln.y[0,-1] ) / (soln.y[1,-1] - soln.y[1,-2])


def get_drange(theta, force, terminate):
     return derivative(get_range, x0=theta, dx=dtheta, args=(v0, force, terminate))
                       

optimum_angle = root_scalar(get_drange, args=(force, terminate), bracket=[theta0, theta1], x0=theta0, x1=theta1, xtol=dtheta, rtol=dtheta)
                       
print(optimum_angle)
